# Debug DAGs

In this notebook, we debug our DAGs without running them through Apache Airflow, that is, by pasing them to EMR cluster directly.

In [1]:
import sys
sys.path.append('airflow/dags/lib')
import emrspark_lib as emrs
import configparser
import time
import boto3

import logging
import os
import json

logger = logging.getLogger()
logger.setLevel(logging.INFO)

config = configparser.ConfigParser()
config.read('airflow/config.cfg')

CLUSTER_NAME = config['AWS']['CLUSTER_NAME']
VPC_ID = config['AWS']['VPC_ID']
SUBNET_ID = config['AWS']['SUBNET_ID']

if config['App']['STOCKS'] == '':
    STOCKS = []
else:
    STOCKS = json.loads(config.get('App', 'STOCKS').replace("'", '"'))

ec2, emr, iam = emrs.get_boto_clients(config['AWS']['REGION_NAME'], config=config)

if VPC_ID == '':
    VPC_ID = emrs.get_first_available_vpc(ec2)

if SUBNET_ID == '':
    SUBNET_ID = emrs.get_first_available_subnet(ec2, VPC_ID)

In [2]:
master_sg_id = emrs.create_security_group(ec2, '{}SG'.format(CLUSTER_NAME),
    'Master SG for {}'.format(CLUSTER_NAME), VPC_ID)
slave_sg_id = emrs.create_security_group(ec2, '{}SlaveSG'.format(CLUSTER_NAME),
    'Slave SG for {}'.format(CLUSTER_NAME), VPC_ID)

keypair = emrs.create_key_pair(ec2, '{}_pem'.format(CLUSTER_NAME))

emrs.create_default_roles(iam)
emrs.wait_for_roles(iam)

cluster_id = emrs.create_emr_cluster(emr, CLUSTER_NAME,
                master_sg_id,
                slave_sg_id,
                keypair['KeyName'], SUBNET_ID,
                release_label='emr-5.28.1',
                num_core_nodes=3)
cluster_dns = emrs.get_cluster_dns(emr, cluster_id)

INFO:root:Found Security Group: sg-031830023db36a998 in vpc vpc-0a0f40087989cc73e (us-east-1).
INFO:root:Found Security Group: sg-01ee208f3283ea93b in vpc vpc-0a0f40087989cc73e (us-east-1).
INFO:botocore.vendored.requests.packages.urllib3.connectionpool:Starting new HTTPS connection (1): iam.amazonaws.com
INFO:root:Role EMR_EC2_DefaultRole is ready
INFO:root:Role EMR_DefaultRole is ready
INFO:root:Instance Profile EMR_EC2_DefaultRole is ready
INFO:botocore.vendored.requests.packages.urllib3.connectionpool:Starting new HTTPS connection (1): elasticmapreduce.us-east-1.amazonaws.com


**The cells below do not need to be run sequentially.**

## Debugging Console - start

### Locally

In [ ]:
from pyspark.sql import SparkSession

spark = SparkSession \
        .builder \
        .config("spark.jars.packages", "org.apache.hadoop:hadoop-aws:2.7.0") \
        .getOrCreate()
#         .config("spark.eventLog.enabled", "true") \
#         .config("spark.eventLog.dir" "test_data/spark-logs") \

import pandas as pd
from pyspark.sql import functions as F

sc = spark.sparkContext
sc._jsc.hadoopConfiguration().set("fs.s3a.access.key", config['AWS']['AWS_ACCESS_KEY_ID'])
sc._jsc.hadoopConfiguration().set("fs.s3a.secret.key", config['AWS']['AWS_SECRET_ACCESS_KEY'])

In [ ]:
sdf = spark.read.csv('s3a://short-interest-effect/data/raw/short_interests_nasdaq', header=True)

In [ ]:
from pyspark.sql.functions import monotonically_increasing_id
df = df.withColumn("index", monotonically_increasing_id())

# Query with the index
tail = sqlContext.sql("""SELECT * FROM df ORDER BY index DESC limit 5""")
tail.show()

#### On cluster

In [ ]:
session_headers = emrs.create_spark_session(cluster_dns)

In [ ]:
url = emrs.spark_url(cluster_dns, location='/sessions')
url

In [ ]:
import requests
response = requests.get(emrs.spark_url(cluster_dns, location='/sessions'))
spark_sessions = response.json()['sessions']
print(spark_sessions)

In [ ]:
job_response_headers = {'Date': 'Mon, 27 Jan 2020 15:37:51 GMT', 'Content-Type': 'application/json', 'Content-Encoding': 'gzip', 'Location': '/sessions/31/statements/0', 'Transfer-Encoding': 'chunked', 'Server': 'Jetty(9.2.16.v20160414)'}
emrs.track_spark_job(cluster_dns, job_response_headers)

## Debugging Console - end

## Pull Stock Info

In [ ]:
args = {
    'AWS_ACCESS_KEY_ID': config['AWS']['AWS_ACCESS_KEY_ID'],
    'AWS_SECRET_ACCESS_KEY': config['AWS']['AWS_SECRET_ACCESS_KEY'],
    'START_DATE': config['App']['START_DATE'],
    'URL_NASDAQ': 'https://old.nasdaq.com/screening/companies-by-name.aspx?letter=0&exchange=nasdaq&render=download',
    'URL_NYSE': 'https://old.nasdaq.com/screening/companies-by-name.aspx?letter=0&exchange=nyse&render=download',
    'DB_HOST': 's3a://short-interest-effect',
    'TABLE_STOCK_INFO_NASDAQ': '/data/raw/stock_info_nasdaq',
    'TABLE_STOCK_INFO_NYSE': '/data/raw/stock_info_nyse',
}

In [ ]:
emrs.kill_all_spark_sessions(cluster_dns)
session_headers = emrs.create_spark_session(cluster_dns)
emrs.wait_for_spark(cluster_dns, session_headers)
job_response_headers = emrs.submit_spark_job_from_file(
        cluster_dns, session_headers,
        'airflow/dags/etl/pull_stock_info.py',
        args=args,
        commonpath='airflow/dags/etl/common.py',
        helperspath='airflow/dags/etl/helpers.py'
)
final_status, logs = emrs.track_spark_job(cluster_dns, job_response_headers)
emrs.kill_spark_session(cluster_dns, session_headers)
for line in logs:
    logging.info(line)
    if '(FAIL)' in str(line):
        logging.error(line)
        raise Exception("ETL process fails.")

In [ ]:
print(cluster_id)
print(master_sg_id)

## Pull Short Interests

In [3]:
args_si = {
    'START_DATE': config['App']['START_DATE'],
    'QUANDL_API_KEY': config['Quandl']['API_KEY'],
    'YESTERDAY_DATE': '2020-02-04',
#     'LIMIT': config['App']['STOCK_LIMITS'],
#     'STOCKS': STOCKS,
    'LIMIT': None,
    'STOCKS': [],
    'AWS_ACCESS_KEY_ID': config['AWS']['AWS_ACCESS_KEY_ID'],
    'AWS_SECRET_ACCESS_KEY': config['AWS']['AWS_SECRET_ACCESS_KEY'],
    'DB_HOST': config['App']['DB_HOST'],
    'TABLE_STOCK_INFO_NASDAQ': config['App']['TABLE_STOCK_INFO_NASDAQ'],
    'TABLE_STOCK_INFO_NYSE': config['App']['TABLE_STOCK_INFO_NYSE'],
    'TABLE_SHORT_INTERESTS_NASDAQ': config['App']['TABLE_SHORT_INTERESTS_NASDAQ'],
    'TABLE_SHORT_INTERESTS_NYSE': config['App']['TABLE_SHORT_INTERESTS_NYSE'],
}

In [6]:
emrs.kill_all_spark_sessions(cluster_dns)
session_headers = emrs.create_spark_session(cluster_dns)
emrs.wait_for_spark(cluster_dns, session_headers)
job_response_headers = emrs.submit_spark_job_from_file(
        cluster_dns, session_headers,
        'airflow/dags/etl/pull_short_interests.py',
        args=args_si,
        commonpath='airflow/dags/etl/common.py',
        helperspath='airflow/dags/etl/helpers.py'
)
final_status, logs = emrs.track_spark_job(cluster_dns, job_response_headers, sleep_seconds=10)
emrs.kill_spark_session(cluster_dns, session_headers)
for line in logs:
    logging.info(line)
    if '(FAIL)' in str(line):
        logging.error(line)
        raise Exception("ETL process fails.")

INFO:root:Killing all spark sessions
INFO:root:Killed idle spark session id 1
INFO:root:Sent spark session creation command to http://ec2-54-89-71-25.compute-1.amazonaws.com:8998/sessions
INFO:root:Response headers: {'Date': 'Fri, 07 Feb 2020 14:45:30 GMT', 'Content-Type': 'application/json;charset=utf-8', 'Content-Encoding': 'gzip', 'Location': '/sessions/2', 'Transfer-Encoding': 'chunked', 'Server': 'Jetty(9.3.24.v20180605)'}
INFO:root:{'id': 2, 'name': None, 'appId': None, 'owner': None, 'proxyUser': None, 'state': 'starting', 'kind': 'pyspark', 'appInfo': {'driverLogUrl': None, 'sparkUiUrl': None}, 'log': ['stdout: ', '\nstderr: ', '\nYARN Diagnostics: ']}
INFO:root:Session headers: {'Date': 'Fri, 07 Feb 2020 14:45:30 GMT', 'Content-Type': 'application/json;charset=utf-8', 'Content-Encoding': 'gzip', 'Location': '/sessions/2', 'Transfer-Encoding': 'chunked', 'Server': 'Jetty(9.3.24.v20180605)'}
INFO:root:Spark session status: starting
INFO:root:Spark session status: starting
INFO:r

ValueError: Stopped because the final job status was "error".

## Pull Prices

In [ ]:
args_p = {
    'START_DATE': config['App']['START_DATE'],
    'QUANDL_API_KEY': config['Quandl']['API_KEY'],
    'YESTERDAY_DATE': '2020-02-04',
    'LIMIT': config['App']['STOCK_LIMITS'],
    'STOCKS': STOCKS,
    'AWS_ACCESS_KEY_ID': config['AWS']['AWS_ACCESS_KEY_ID'],
    'AWS_SECRET_ACCESS_KEY': config['AWS']['AWS_SECRET_ACCESS_KEY'],
    'DB_HOST': config['App']['DB_HOST'],
    'TABLE_STOCK_INFO_NASDAQ': config['App']['TABLE_STOCK_INFO_NASDAQ'],
    'TABLE_STOCK_INFO_NYSE': config['App']['TABLE_STOCK_INFO_NYSE'],
    'TABLE_STOCK_PRICES': config['App']['TABLE_STOCK_PRICES'],
    'URL': "http://app.quotemedia.com/quotetools/getHistoryDownload.csv?&webmasterId=501&startDay={sd}&startMonth={sm}&startYear={sy}&endDay={ed}&endMonth={em}&endYear={ey}&isRanged=true&symbol={sym}",
}

In [ ]:
emrs.kill_all_spark_sessions(cluster_dns)
session_headers = emrs.create_spark_session(cluster_dns)
emrs.wait_for_spark(cluster_dns, session_headers)
job_response_headers = emrs.submit_spark_job_from_file(
        cluster_dns, session_headers,
        'airflow/dags/etl/pull_prices.py',
        args=args_p,
        commonpath='airflow/dags/etl/common.py',
        helperspath='airflow/dags/etl/helpers.py'
)
final_status, logs = emrs.track_spark_job(cluster_dns, job_response_headers, sleep_seconds=5)

## Quality-check - Short Interests

In [ ]:
args_qs = {
            'AWS_ACCESS_KEY_ID': config['AWS']['AWS_ACCESS_KEY_ID'],
            'AWS_SECRET_ACCESS_KEY': config['AWS']['AWS_SECRET_ACCESS_KEY'],
            'YESTERDAY_DATE': '2020-02-03',
            'STOCKS': STOCKS,
            'DB_HOST': config['App']['DB_HOST'],
            'TABLE_SHORT_INTERESTS_NASDAQ': config['App']['TABLE_SHORT_INTERESTS_NASDAQ'],
            'TABLE_SHORT_INTERESTS_NYSE': config['App']['TABLE_SHORT_INTERESTS_NYSE'],
        }

In [ ]:
emrs.kill_all_spark_sessions(cluster_dns)
session_headers = emrs.create_spark_session(cluster_dns)
emrs.wait_for_spark(cluster_dns, session_headers)
job_response_headers = emrs.submit_spark_job_from_file(
        cluster_dns, session_headers,
        'airflow/dags/etl/pull_short_interests_quality.py',
        args=args_qs,
        commonpath='airflow/dags/etl/common.py',
        helperspath='airflow/dags/etl/helpers.py'
)
final_status, logs = emrs.track_spark_job(cluster_dns, job_response_headers, sleep_seconds=10)

## Quality-check - Prices

In [ ]:
args_qp = {
            'AWS_ACCESS_KEY_ID': config['AWS']['AWS_ACCESS_KEY_ID'],
            'AWS_SECRET_ACCESS_KEY': config['AWS']['AWS_SECRET_ACCESS_KEY'],
            'YESTERDAY_DATE': '2020-02-03',
            'STOCKS': STOCKS,
            'DB_HOST': config['App']['DB_HOST'],
            'TABLE_STOCK_INFO_NASDAQ': config['App']['TABLE_STOCK_INFO_NASDAQ'],
            'TABLE_STOCK_INFO_NYSE': config['App']['TABLE_STOCK_INFO_NYSE'],
            'TABLE_STOCK_PRICES': config['App']['TABLE_STOCK_PRICES'],
        }

In [ ]:
emrs.kill_all_spark_sessions(cluster_dns)
session_headers = emrs.create_spark_session(cluster_dns)
emrs.wait_for_spark(cluster_dns, session_headers)
job_response_headers = emrs.submit_spark_job_from_file(
        cluster_dns, session_headers,
        'airflow/dags/etl/pull_prices_quality.py',
        args=args_qp,
        commonpath='airflow/dags/etl/common.py',
        helperspath='airflow/dags/etl/helpers.py'
)
final_status, logs = emrs.track_spark_job(cluster_dns, job_response_headers, sleep_seconds=10)
emrs.kill_spark_session(cluster_dns, session_headers)
for line in logs:
    logging.info(line)
    if '(FAIL)' in str(line):
        logging.error(line)
        raise Exception("ETL process fails.")

## Combine Data

In [ ]:
args_c = {
            'YESTERDAY_DATE': '2020-12-10',
            'AWS_ACCESS_KEY_ID': config['AWS']['AWS_ACCESS_KEY_ID'],
            'AWS_SECRET_ACCESS_KEY': config['AWS']['AWS_SECRET_ACCESS_KEY'],
            'DB_HOST': config['App']['DB_HOST'],
            'TABLE_STOCK_PRICES': config['App']['TABLE_STOCK_PRICES'],
            'TABLE_SHORT_INTERESTS_NASDAQ': config['App']['TABLE_SHORT_INTERESTS_NASDAQ'],
            'TABLE_SHORT_INTERESTS_NYSE': config['App']['TABLE_SHORT_INTERESTS_NYSE'],
            'TABLE_SHORT_ANALYSIS': config['App']['TABLE_SHORT_ANALYSIS'],
        }

In [ ]:
logger.setLevel(logging.INFO)
emrs.kill_all_spark_sessions(cluster_dns)
session_headers = emrs.create_spark_session(cluster_dns)
emrs.wait_for_spark(cluster_dns, session_headers)
job_response_headers = emrs.submit_spark_job_from_file(
        cluster_dns, session_headers,
        'airflow/dags/etl/combine.py',
        args=args_c,
        commonpath='airflow/dags/etl/common.py',
        helperspath='airflow/dags/etl/helpers.py'
)
final_status, logs = emrs.track_spark_job(cluster_dns, job_response_headers)
emrs.kill_spark_session(cluster_dns, session_headers)
for line in logs:
    logging.info(line)
    if '(FAIL)' in str(line):
        logging.error(line)
        raise Exception("ETL process fails.")

## Convert to Quantopian

In [ ]:
args_cq = {
            'AWS_ACCESS_KEY_ID': config['AWS']['AWS_ACCESS_KEY_ID'],
            'AWS_SECRET_ACCESS_KEY': config['AWS']['AWS_SECRET_ACCESS_KEY'],
            'DB_HOST': config['App']['DB_HOST'],
            'TABLE_SHORT_ANALYSIS': config['App']['TABLE_SHORT_ANALYSIS'],
            'TABLE_SHORT_ANALYSIS_QUANTOPIAN': config['App']['TABLE_SHORT_ANALYSIS_QUANTOPIAN']
        }

In [ ]:
emrs.kill_all_spark_sessions(cluster_dns)
session_headers = emrs.create_spark_session(cluster_dns)
emrs.wait_for_spark(cluster_dns, session_headers)
job_response_headers = emrs.submit_spark_job_from_file(
        cluster_dns, session_headers,
        'airflow/dags/etl/prepare_for_quantopian.py',
        args=args_cq,
        commonpath='airflow/dags/etl/common.py',
        helperspath='airflow/dags/etl/helpers.py'
)
final_status, logs = emrs.track_spark_job(cluster_dns, job_response_headers, sleep_seconds=10)
emrs.kill_spark_session(cluster_dns, session_headers)
for line in logs:
    logging.info(line)
    if '(FAIL)' in str(line):
        logging.error(line)
        raise Exception("ETL process fails.")
        
# Update files to public readable:

bucket = config['App']['DB_HOST'].split('/')[-1]

key = config['App']['TABLE_SHORT_ANALYSIS'][1:]+'.csv'
(boto3
 .session
 .Session(region_name='us-east-1')
 .resource('s3')
 .Object(bucket, key)
 .copy_from(CopySource={'Bucket': bucket,
                        'Key': key},
            MetadataDirective="REPLACE",
            ContentType="text/csv")
)
(boto3
 .session
 .Session(region_name='us-east-1')
 .resource('s3')
 .Object(bucket, key)
 .Acl()
 .put(ACL='public-read'))

key = config['App']['TABLE_SHORT_ANALYSIS_QUANTOPIAN'][1:]+'.csv'
(boto3
 .session
 .Session(region_name='us-east-1')
 .resource('s3')
 .Object(bucket, key)
 .copy_from(CopySource={'Bucket': bucket,
                        'Key': key},
            MetadataDirective="REPLACE",
            ContentType="text/csv")
)
(boto3
 .session
 .Session(region_name='us-east-1')
 .resource('s3')
 .Object(bucket, key)
 .Acl()
 .put(ACL='public-read'))

## Delete Cluster

In [ ]:
emrs.delete_cluster(emr, cluster_id)

INFO:botocore.vendored.requests.packages.urllib3.connectionpool:Resetting dropped connection: elasticmapreduce.us-east-1.amazonaws.com
INFO:root:Cluster j-2445HR9SR059T has not been terminated (Current cluster state: TERMINATING). waiting until the status is TERMINATED...
INFO:botocore.vendored.requests.packages.urllib3.connectionpool:Resetting dropped connection: elasticmapreduce.us-east-1.amazonaws.com
INFO:root:Cluster j-2445HR9SR059T has not been terminated (Current cluster state: TERMINATING). waiting until the status is TERMINATED...
